# Titanic Data Science Solutions

<h2> This notebook is a companion to the book <a href='https://www.amazon.com/Data-Science-Solutions-Startup-Workflow/dp/1520545312' target='blank'>Data Science Solutions.</a></h2>

The notebook walks us through a typical workflow for solving data science competitions at sites like Kaggle.

There are several excellent notebooks to study data science competition entries. However many will skip some of the explanation on how the solution is developed as these notebooks are developed by experts for experts. The objective of this notebook is to follow a step-by-step workflow, explaining each step and rationale for every decision we take during solution development.

<h2>Workflow stages</h2>

The competition solution workflow goes through seven stages described in the Data Science Solutions book.